In [ ]:
import pandas as pd
path = 'data/FT_Camp_5/Train.csv'
data = pd.read_csv(path)
data.set_index('stockcode',inplace=True)
data.head()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold,cross_val_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [ ]:
X=data.iloc[:,0:34]
y=data.fake

In [ ]:
x_train,y_train,x_test,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
params_dict={
    'learning_rate':0.129154966501488,
    'n_estimators':50,
    'gamma':0.0,
    'max_depth':8,
    'min_child_weight':1,
    'colsample_bytree':0.9,
    'colsample_bylevel':1.0,
    'subsample':0.7,
    'reg_lambda':0.0,
    'reg_alpha':0.4,
    'seed':42
}
clf=xgb.XGBClassifier(**params_dict)
# cv_params={'n_estimators':np.linspace(50,150,11,dtype=int)}  50
# cv_params={'max_depth':np.linspace(1,10,10,dtype=int)} 8
# cv_params={'min_child_weight':np.linspace(1,10,10,dtype=int)} 1
# cv_params={'gamma':np.linspace(0,0.1,11)}
# cv_params={'subsample':np.linspace(0,1,11)} 0.7
# cv_params={'colsample_bytree':np.linspace(0,1,11)[1:]} 0.9
# cv_params={'colsample_bylevel':np.linspace(0.9,1,11)[1:]} 1.0
# cv_params={'re_lambda':np.linspace(0,1,11)} 0.0
# cv_params={'reg_alpha':np.linspace(0,1,11)}  0.4
cv_params={'learning_rate':np.logspace(-1,0,10)}
gs = GridSearchCV(clf,cv_params,verbose=2,refit=True,cv=5,n_jobs=-1)
gs.fit(X,y)
print('best value:',gs.best_params_)
print('socre:',gs.best_score_)

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score
def f1_cv(model,X,y):
    f1=cross_val_score(model,X,y,scoring='f1_macro',cv=5)
    return f1
default_xgb=xgb.XGBClassifier()
score=f1_cv(default_xgb,X,y)
print('default:{:.4f}({:.4f})'.format(score.mean(),score.std()))


In [ ]:
tuned_xgb=xgb.XGBClassifier(**params_dict)
score_t=f1_cv(tuned_xgb,X,y)
print('tuned:{:.4f}({:.4f})'.format(score_t.mean(),score_t.std()))

In [ ]:
test_path='data/FT_Camp_5/X_test.csv'
test_data=pd.read_csv(test_path)
test_data.set_index('stockcode',inplace=True)

In [ ]:
xgb_pred=xgb.XGBClassifier(**params_dict)
xgb_pred.fit(X,y)
test_data['fake']=xgb_pred.predict(test_data)

In [ ]:
save_=test_data.fake
save_.to_csv('unsample_xgb.csv',sep=',',header=True,index=False,encoding='UTF-8')